In [218]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

## Lendo dados do DataSet

In [219]:
# Leitura dos Dataset
df = pd.read_csv("games.csv")
df.shape

(51490, 61)

In [220]:
#remoção das colunas desnecessárias
df = df.drop(['gameId', 'creationTime'], axis=1)
df

,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,1949,9,1,2,1,1,1,1,2,8,...,5,0,0,1,1,114,67,43,16,51
1,1851,9,1,1,1,1,0,1,1,119,...,2,0,0,0,0,11,67,238,51,420
2,1493,9,1,2,1,1,1,2,0,18,...,2,0,0,1,0,157,238,121,57,28
3,1758,9,1,1,1,1,1,1,0,57,...,0,0,0,0,0,164,18,141,40,51
4,2094,9,1,2,1,1,1,1,0,19,...,3,0,0,1,0,86,11,201,122,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51485,1944,9,2,1,2,2,0,2,0,19,...,10,2,0,4,0,55,-1,90,238,157
51486,3304,9,2,1,1,2,2,2,2,90,...,11,7,4,4,1,157,55,119,154,105
51487,2156,9,2,2,2,2,0,1,0,141,...,10,2,0,2,0,113,122,53,11,157
51488,1475,9,2,2,2,2,0,2,0,121,...,11,3,0,1,0,154,39,51,90,114


In [206]:

df = df.fillna(0)
df

,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,1949,9,1,2,1,1,1,1,2,8,...,5,0,0,1,1,114,67,43,16,51
1,1851,9,1,1,1,1,0,1,1,119,...,2,0,0,0,0,11,67,238,51,420
2,1493,9,1,2,1,1,1,2,0,18,...,2,0,0,1,0,157,238,121,57,28
3,1758,9,1,1,1,1,1,1,0,57,...,0,0,0,0,0,164,18,141,40,51
4,2094,9,1,2,1,1,1,1,0,19,...,3,0,0,1,0,86,11,201,122,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51485,1944,9,2,1,2,2,0,2,0,19,...,10,2,0,4,0,55,-1,90,238,157
51486,3304,9,2,1,1,2,2,2,2,90,...,11,7,4,4,1,157,55,119,154,105
51487,2156,9,2,2,2,2,0,1,0,141,...,10,2,0,2,0,113,122,53,11,157
51488,1475,9,2,2,2,2,0,2,0,121,...,11,3,0,1,0,154,39,51,90,114


## Lendo os Dados do Arquivos auxiliares

* summoner_spell_info.json
* champion_info_2.json

In [221]:
champions_info_df = pd.read_json('champions_info_modified.json')


# print(champions_info_df.head())
# print(summoner_spell_info_df.head())
champions_info_df = champions_info_df['data'] 
champions_info_transposed = champions_info_df.transpose()
print(champions_info_transposed)
# Lendo o arquivo summoner_spell_info.json
summoner_spell_info_df = pd.read_json('summoner_spell_info.json')

-1      {'tags': [], 'title': 'None', 'id': -1, 'key':...
 1      {'tags': ['Mage'], 'title': 'the Dark Child', ...
 10     {'tags': ['Fighter', 'Support'], 'title': 'The...
 101    {'tags': ['Mage', 'Assassin'], 'title': 'the M...
 102    {'tags': ['Fighter', 'Tank'], 'title': 'the Ha...
                              ...                        
 91     {'tags': ['Assassin', 'Fighter'], 'title': 'th...
 92     {'tags': ['Fighter', 'Assassin'], 'title': 'th...
 96     {'tags': ['Marksman', 'Mage'], 'title': 'the M...
 98     {'tags': ['Tank'], 'title': 'the Eye of Twilig...
 99     {'tags': ['Mage', 'Support'], 'title': 'the La...
Name: data, Length: 139, dtype: object


In [222]:
#Pegando a coluna a ser analisada(Quem venceu)
x = df.drop(['winner'], axis=1).values 
y = df['winner']

In [223]:
# Separando Teste(20) e Treino(80)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [214]:
# Escalonando


# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(trainData)
# X_test_scaled = scaler.transform(testData)

In [224]:

# Modelos
# Decision Tree
# KNN
# Regrassão Logistica
# Random Forest
# Redes Neurais
# 
# Criando o objeto do modelo KNN




model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train,y_train)
c_pred = model.predict(X_train)
y_pred  = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)


In [ ]:
accuracy